trying to solve 15 kg exercise from slides

get 15 kg of materials from a list of materials with different weights

materials: [1, 1, 1, 1, 1, 1, 5, 5, 7, 7, 9, 9]


In [2]:
import numpy as np
from collections import namedtuple, deque
from tqdm.auto import tqdm
from queue import PriorityQueue
from functools import reduce

In [103]:
items = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
weights = [1, 1, 1, 1, 1, 1, 5, 5, 7, 7, 9, 9]

FINAL_WEIGHT = 15

## try one

In [95]:
# bag of taken/bag of not taken

State = namedtuple('State', ['taken', 'not_taken'])

def compute_weight(state): return sum([weights[x] for x in state.taken])

def goal_check(state): return compute_weight(state) == FINAL_WEIGHT

Depth First

In [98]:
frontier = deque()
state = State(set(), set(items))
frontier.append(state)

counter = 0
current_state = frontier.pop()
current_weight = 0
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for item in current_state[1]: # for item in not taken items
            if current_weight + weights[item] <= FINAL_WEIGHT:
                new_state = State(
                    current_state.taken ^ {item},
                    current_state.not_taken ^ {item},
                )
                frontier.append(new_state)
        current_state = frontier.pop()
        current_weight = compute_weight(current_state)
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

3it [00:00, 3011.71it/s]

Solved in 3 steps (3 tiles)


Breadth First

In [120]:
frontier = deque()
state = State(set(), set(items))
frontier.append(state)

counter = 0
current_state = frontier.popleft()
current_weight = 0
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for item in current_state[1]: # for item in not taken items
            if current_weight + weights[item] <= FINAL_WEIGHT:
                new_state = State(
                    current_state.taken ^ {item},
                    current_state.not_taken ^ {item},
                )
                frontier.append(new_state)
        current_state = frontier.popleft()
        current_weight = compute_weight(current_state)
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

193it [00:00, 38702.46it/s]

Solved in 193 steps (3 tiles)


Greedy Best First

In [8]:
def f(state):
    missing_weight = FINAL_WEIGHT - compute_weight(state)
    return missing_weight

In [72]:
frontier = PriorityQueue()
state = State(set(), set(items))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
current_weight = compute_weight(current_state)
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for item in current_state[1]:
            if current_weight + weights[item] <= FINAL_WEIGHT:
                new_state = State(
                    current_state.taken ^ {item},
                    current_state.not_taken ^ {item},
                )
                frontier.put((f(new_state), new_state))
        _, current_state = frontier.get() # being a priorityQ, the next state will be the one with minimum f
        current_weight = compute_weight(current_state)
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

3it [00:00, ?it/s]

Solved in 3 steps (3 tiles)


## A*

In [73]:
def get_candidates(v, n_to_take, mw):
    lenv = len(v)
    ii = np.array(range(n_to_take))
    candidates = []
    end = False
    cc = [v[iii] for iii in ii]
    ccc = 0
    for c in cc: ccc += weights[c]
    if ccc == mw: return True
    while not end:
        ii[-1] += 1
        for i in range(1, n_to_take):
            if ii[-i] == lenv + 1 - i:
                ii[-i -1] += 1
                for k in range(-i, 0):
                    ii[k] = ii[k - 1] + 1
        if ii[0] == lenv - (n_to_take - 1): end = True
        else:
            cc = [v[iii] for iii in ii]
            ccc = 0
            for c in cc: ccc += weights[c]
            if ccc == mw: return True

    return False

def f_mine(state):
    current_weight = compute_weight(state)
    missing_weight = FINAL_WEIGHT - current_weight

    not_ok = True
    for item in state.not_taken:
        if current_weight + weights[item] <= FINAL_WEIGHT: not_ok = False
    if not_ok: return 999999999

    min_items = 1
    while not get_candidates(list(state.not_taken), min_items, missing_weight): 
        min_items += 1
        if min_items == len(state.not_taken): return 999999999

    return len(state.taken) + min_items

In [74]:
frontier = PriorityQueue()
state = State(set(), set(items))
frontier.put((f_mine(state), state))

counter = 0
ss = []
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            fn = f_mine(new_state)
            frontier.put((fn, new_state))
            ss.append((fn, new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

45185it [00:04, 9839.72it/s] 

Solved in 45,185 steps (5 tiles)


## try two

In [121]:
class Multiset:
    def __init__(self, init_values= None):
        if init_values is not None:
            self.taken, self.not_taken, self.current_weight = init_values

    def first_state(self, values):
        self.taken = {}
        self.not_taken = {}
        self.current_weight = 0
        for v in values:
            try:
                self.taken[v]
                self.not_taken[v] += 1
            except:
                self.taken[v] = 0
                self.not_taken[v] = 1
        print(self.taken)
        print(self.not_taken)
        return self

    def to_take(self):
        tt = []
        for k in self.not_taken.keys():
            if self.not_taken[k] > 0:
                tt.append(k)
        return tt

    def take(self, v):
        self.taken[v] += 1
        self.not_taken[v] -= 1
        self.current_weight += v

    def get_current_weight(self): return self.current_weight

    def get_missing_weight(self): return FINAL_WEIGHT - self.current_weight

    def goal_check(self): return self.current_weight == FINAL_WEIGHT

    def copy(self): return Multiset((self.taken.copy(), self.not_taken.copy(), self.current_weight))

    def __str__(self): return str(f'not_taken: {self.not_taken}\ntaken:     {self.taken}')

# draw from the bag exploration

state = Multiset().first_state(weights)

print(state.to_take())
print(state)

{1: 0, 5: 0, 7: 0, 9: 0}
{1: 6, 5: 2, 7: 2, 9: 2}
[1, 5, 7, 9]
not_taken: {1: 6, 5: 2, 7: 2, 9: 2}
taken:     {1: 0, 5: 0, 7: 0, 9: 0}


Depth first

In [116]:
frontier = deque()
state = Multiset().first_state(weights)
frontier.append(state)

counter = 0
current_state = frontier.pop()
current_weight = 0
with tqdm(total=None) as pbar:
    while not current_state.goal_check():
        counter += 1
        for item in current_state.to_take(): # for item in not taken items
            if current_weight + weights[item] <= FINAL_WEIGHT:
                new_state = current_state.copy()
                new_state.take(item)
                frontier.append(new_state)
        current_state = frontier.pop()
        current_weight = current_state.get_current_weight()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

{1: 0, 5: 0, 7: 0, 9: 0}
{1: 6, 5: 2, 7: 2, 9: 2}


5it [00:00, ?it/s]

Solved in 5 steps (4 tiles)


Breadth First

In [119]:
frontier = deque()
state = Multiset().first_state(weights)
frontier.append(state)

counter = 0
current_state = frontier.popleft()
current_weight = 0
with tqdm(total=None) as pbar:
    while not current_state.goal_check():
        counter += 1
        for item in current_state.to_take(): # for item in not taken items
            if current_weight + weights[item] <= FINAL_WEIGHT:
                new_state = current_state.copy()
                new_state.take(item)
                frontier.append(new_state)
        current_state = frontier.popleft()
        current_weight = current_state.get_current_weight()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

{1: 0, 5: 0, 7: 0, 9: 0}
{1: 6, 5: 2, 7: 2, 9: 2}


27it [00:00, ?it/s]

Solved in 27 steps (4 tiles)


Greedy Best First

In [123]:
frontier = PriorityQueue()
state = Multiset().first_state(weights)
frontier.put((state.get_missing_weight(), state))

counter = 0
_, current_state = frontier.get()
current_weight = current_state.get_current_weight()
with tqdm(total=None) as pbar:
    while not current_state.goal_check():
        counter += 1
        for item in current_state.to_take():
            if current_weight + weights[item] <= FINAL_WEIGHT:
                new_state = current_state.copy()
                new_state.take(item)
                frontier.put((new_state.get_missing_weight(), new_state))
        _, current_state = frontier.get() # being a priorityQ, the next state will be the one with minimum f
        current_weight = current_state.get_current_weight()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

{1: 0, 5: 0, 7: 0, 9: 0}
{1: 6, 5: 2, 7: 2, 9: 2}


5it [00:00, ?it/s]

Solved in 5 steps (4 tiles)


## try three

In [7]:
# single state for each element (taken/not taken)

state = np.zeros(len(weights), dtype= bool)

print(state)

[False False False False False False False False False False False False]
